In [10]:
import math
import numpy as np

In [11]:
#kreujemy strumień
data_stream = np.random.choice([0,1], size=100, replace =True)

In [54]:
window_size = 10 #wielkość okna, jakie możemy przechowywać
timestamp = 0

class DGIM:
    def __init__(self, window_size):
        self.window_size = window_size                        #wielkość okna
        self.buckets = {}                                     #kubełkowy słownik
        self.keys_num = int(math.log(self.window_size))+2     #ile kubełków możemy mieć, związane jest to z wielkością okna, indeksy kubełków
        self.keys_list = []                                   #listy kluczy dla kubełkowego słownika 
        self.cur_timestamp = -1                               #na wejściu -1, ale może być cokolwiek, bo w update i tak przypisujemy nowe

        #tworzenie słownika dla kubełków 
        for i in range(self.keys_num):  #ile kubełków mamy
            key = int(math.pow(2, i))   #klucz to 2 do kolejnej potęgi
            self.keys_list.append(key)  #dodajemy kolejne klucze do listy z kluczami
            self.buckets[key] = []      #dodajemy pustą listę dla każdego klucza w słowniku kubełowym
    
    def update(self, bit, timestamp):            #bit odnosi się tutaj do elementu z nadchodzącej listy, więc bit to w zasadzie[bit...bit]
        timestamp = timestamp % self.window_size #znacznik czasu, modulo wielkość okna, żeby kontrolować ile możemy przechowywać
        self.cur_timestamp = timestamp           #trzymamy aktualny znacznik czasu, bez modulo, bo później operujemy na timestamp
        #print('1', self.buckets)
        if bit == 1:                             #jeśli przyszła '1' to 
            self.buckets[1].append(timestamp)    #dodajemy znacznik czasu do pierwszego kubełka
            for key in self.keys_list:           #sprawdzamy po wszystkich kubełkach
                if len(self.buckets[key]) > 2:   #jeśli długość listy dla danego klucza kubełowego jest większa od 2
                    self.buckets[key].pop(0)     #to wyrzucamy pierwszy element(najstarszy)
                    timestamp = self.buckets[key].pop(0) #usuwamy drugi i oznaczamy go jako czas
                    if key != self.keys_list[-1]: #jeśli klucz nie znajduje się to go dodaj
                        self.buckets[key * 2].append(timestamp)
                        #print('2',self.buckets)
                else:
                    break
                
    def estimation(self):
        sum = 0 
        last = list(self.buckets.keys())[-1]
        for key in list(self.buckets.keys())[:-1]:
            sum =+ key*len(self.buckets[key])
        for value in range(len(self.buckets[last])):
            if sum + last <= self.window_size:
                sum += last
            else:
                sum += 0.5*last
                break
        print("Wyestymowana ilość jedynek w ostatnich {} bitach to: {}".format(self.window_size, sum))

In [56]:
dgim = DGIM(100)
for i in range(len(data_stream)):
    dgim.update(data_stream[i],i)
dgim.estimation()

Wyestymowana ilość jedynek w ostatnich 100 bitach to: 32
